In [ ]:
from model import *
from networks import networks
import copy

In [ ]:
COMPARISON_TABLE_PATH = 'comparison_table.csv'

In [ ]:
def run_on_hardware(layers, num_out_vec, proto_length_vec, num_proto_vec, num_subspace_vec, memory):
    params = {
        "NUM_OUT_VEC": num_out_vec,
        "PROTO_LENGTH_VEC": proto_length_vec,
        "NUM_PROTO_VEC": num_proto_vec,
        # "NUM_PROTO_LAYER": N_p,
        "NUM_SUBSPACE_VEC": num_subspace_vec,
        # "PROTO_LENGTH_LAYER": L_s,
    }

    total_cycles = 0
    initialization_cycles = 0
    calculation_cycles_proto = 0
    calculation_cycles_pq = 0
    calculation_cycles_full = 0
    cycles_dla = 0
    for layer in layers:
        params.update(layer[3])
        input_shape = layer[0]
        filter_shape = layer[1]
        strides = layer[2]

        result = simulate_layer(
            INPUT_SHAPE             = input_shape,
            FILTER_SHAPE            = filter_shape,
            STRIDES                 = strides,
            FREQUENCY_DLA           = freq_dla,
            FREQUENCY_PQ            = freq_pq,
            MODEL_FREQUENCY         = model_frequency,
            OVERLAPPING_IN_CALC     = overlapping_calculation_with_loading_pq,
            USE_CUSTOM_EQUATION     = dla_custom_equation,
            MEMORY_TYPE             = memory,
            **params
        )

        total_cycles+= result['PQ']['Ncycles']
        initialization_cycles+= result['PQ']['Ncycles_initialization']
        calculation_cycles_proto+= result['PQ']['Ncycles_calculation_proto']
        calculation_cycles_pq+= result['PQ']['Ncycles_calculation_pq']
        calculation_cycles_full+= result['PQ']['Ncycles_calculation']
        cycles_dla+= result['DLA']['Ncycles']

    return initialization_cycles, calculation_cycles_proto, calculation_cycles_pq, calculation_cycles_full, total_cycles, cycles_dla

In [ ]:
configs = {
    'ResNet-20': [
        {'PROTO_LENGTH_LAYER': 3, 'NUM_PROTO_LAYER': 64},
        {'PROTO_LENGTH_LAYER': 9, 'NUM_PROTO_LAYER': 64},
        {'PROTO_LENGTH_LAYER': 9, 'NUM_PROTO_LAYER': 8},
        {'PROTO_LENGTH_LAYER': 9, 'NUM_PROTO_LAYER': 16},
        {'PROTO_LENGTH_LAYER': 9, 'NUM_PROTO_LAYER': 8, 'PROTO_TABLE_ENTRY': 6, 'PQ_TABLE_ENTRY': 5},
        {'PROTO_LENGTH_LAYER': 9, 'NUM_PROTO_LAYER': 16, 'PROTO_TABLE_ENTRY': 6, 'PQ_TABLE_ENTRY': 5},
    ],
    'kws': [
        {'PROTO_LENGTH_LAYER': 8, 'NUM_PROTO_LAYER': 8},
        {'PROTO_LENGTH_LAYER': 4, 'NUM_PROTO_LAYER': 16},
        {'PROTO_LENGTH_LAYER': 8, 'NUM_PROTO_LAYER': 8, 'PROTO_TABLE_ENTRY': 2, 'PQ_TABLE_ENTRY': 6},
        {'PROTO_LENGTH_LAYER': 4, 'NUM_PROTO_LAYER': 16, 'PROTO_TABLE_ENTRY': 2, 'PQ_TABLE_ENTRY': 6},
    ],
    'EMNIST': [
        {'PROTO_LENGTH_LAYER': 4, 'NUM_PROTO_LAYER': 12},
        {'PROTO_LENGTH_LAYER': 8, 'NUM_PROTO_LAYER': 8},
        {'PROTO_LENGTH_LAYER': 8, 'NUM_PROTO_LAYER': 8, 'PROTO_TABLE_ENTRY': 5, 'PQ_TABLE_ENTRY': 5},
        {'PROTO_LENGTH_LAYER': 4, 'NUM_PROTO_LAYER': 12, 'PROTO_TABLE_ENTRY': 5, 'PQ_TABLE_ENTRY': 5},

    ]
}

In [ ]:
import csv

# FILEPATH: /home/afa55/Projects/product_quantization_hls/analytical_model/print_comparison_table.ipynb
with open(COMPARISON_TABLE_PATH, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Network', 'Memory', 'Config', 'num_out_vec','proto_length_vec','num_proto_vec','num_subspace_vec','Initialization Cycles', 'Calculation Cycles Proto', 'Calculation Cycles PQ', 'Calculation Cycles', 'Total Cycles', 'DLA Cycles'])
    
    for memory in ['DDR', 'HBM']:
        for network_name, vals in configs.items():
            network = copy.deepcopy(networks[network_name])
            for val in vals:
                for layer in network:
                    layer[3].update(val)
                proto_length_vec = min(16, val['PROTO_LENGTH_LAYER'])
                num_proto_vec = min(16, val['NUM_PROTO_LAYER'])
                num_subspace_vecs = [16, 32, 64]
                num_out_vecs = [32,64]
                for num_out_vec in num_out_vecs:
                    for num_subspace_vec in num_subspace_vecs:
                        pq_init, pq_calc_proto, pq_calc_pq, pq_calc, pq_total, dla = run_on_hardware(network, num_out_vec, proto_length_vec, num_proto_vec, num_subspace_vec, memory)
                        writer.writerow([network_name, memory, val, num_out_vec, proto_length_vec, num_proto_vec, num_subspace_vec, pq_init, pq_calc_proto, pq_calc_pq, pq_calc, pq_total, dla])

print(f"Comparison table has been written to {COMPARISON_TABLE_PATH}")
